In [28]:
from dotenv import load_dotenv
from dataclasses import dataclass
from langchain.agents import AgentState, create_agent
from langchain.tools import tool, ToolRuntime
from langgraph.types import Command
from langchain.messages import ToolMessage
from langchain.agents.middleware import wrap_model_call, dynamic_prompt, HumanInTheLoopMiddleware
from langchain.agents.middleware import ModelRequest, ModelResponse
from typing import Callable
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import InMemorySaver
import os

load_dotenv()


OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
if not OPENROUTER_API_KEY:
    raise EnvironmentError("Установите OPENROUTER_API_KEY в файле .env")


llm = ChatOpenAI(
    model="google/gemini-3-flash-preview",
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY,
    temperature=0.0
)

In [29]:
from dataclasses import dataclass

@dataclass
class EmailContext:
    email_address: str = "julie@example.com"
    password: str = "password123"

In [30]:
from langchain.agents import AgentState

class AuthenticatedState(AgentState):
    authenticated: bool

In [31]:
@tool
def check_inbox() -> str:
    """Check the inbox for recent emails"""
    return """
    Hi Julie, 
    I'm going to be in town next week and was wondering if we could grab a coffee?
    - best, Jane (jane@example.com)
    """

@tool
def send_email(to: str, subject: str, body: str) -> str:
    """Send an response email"""
    return f"Email sent to {to} with subject {subject} and body {body}"

@tool
def authenticate(email: str, password: str, runtime: ToolRuntime) -> Command:
    """Authenticate the user with the given email and password"""
    if email == runtime.context.email_address and password == runtime.context.password:
        return Command(update={
            "authenticated": True, 
            "messages": [ToolMessage(
                "Successfully authenticated", 
                tool_call_id=runtime.tool_call_id)]
        })
    else:
        return Command(update={
            "authenticated": False,
            "messages": [ToolMessage(
                "Authentication failed", 
                tool_call_id=runtime.tool_call_id)]
        })

In [32]:
@wrap_model_call
def dynamic_tool_call(request: ModelRequest, 
handler: Callable[[ModelRequest], ModelResponse]) -> ModelResponse:

    """Allow read inbox and send email tools only if user provides correct email and password"""

    authenticated = request.state.get("authenticated")
    
    if authenticated:
        tools = [check_inbox, send_email]
    else:
        tools = [authenticate]

    request = request.override(tools=tools) 
    return handler(request)

In [33]:
authenticated_prompt = "You are a helpful assistant that can check the inbox and send emails."
unauthenticated_prompt = "You are a helpful assistant that can authenticate users."

@dynamic_prompt
def dynamic_prompt(request: ModelRequest) -> str:
    """Generate system prompt based on authentication status"""
    authenticated = request.state.get("authenticated")

    if authenticated:
        return authenticated_prompt
    else:
        return unauthenticated_prompt

In [34]:
agent = create_agent(
    model=llm,
    tools=[authenticate, check_inbox, send_email],
    checkpointer=InMemorySaver(),
    state_schema=AuthenticatedState,
    context_schema=EmailContext,
    middleware=[
        dynamic_tool_call, 
        dynamic_prompt,
        HumanInTheLoopMiddleware(
            interrupt_on={
                "authenticate": False,
                "check_inbox": False,
                "send_email": True,
            })
        ]
    )

In [35]:
from langchain.messages import HumanMessage


config = {"configurable": {"thread_id": "1"}}

# 1. Аутентификация (у тебя уже работает!)
auth_response = agent.invoke(
    {"messages": [HumanMessage(
        content="My email is julie@example.com and my password is password123"
    )]},
    context=EmailContext(),
    config=config
)
print("Auth result:", auth_response["messages"][-1].content)

# 2. Чёткая команда на отправку письма
response = agent.invoke(
    {"messages": [HumanMessage(
        content="Ответь на письмо от Jane, что я согласен встретиться на кофе."
    )]},
    context=EmailContext(),
    config=config
)

# 3. Теперь должно быть прерывание!
if "__interrupt__" in response:
    print("📄 Черновик письма:")
    print(response["__interrupt__"][0].value["action_requests"][0]["args"]["body"])
    
    # 4. Подтверждаем отправку
    final_response = agent.invoke(
        Command(resume={"decisions": [{"type": "approve"}]}),
        config=config
    )
    print("\n✅ Результат:", final_response["messages"][-1].content)
else:
    print("❗ Нет прерывания. Агент ответил:")
    print(response["messages"][-1].content)

c:\Users\ryash\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydantic\main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `none` - serialized value may not be as expected [input_value=EmailContext(email_addres... password='password123'), input_type=EmailContext])
  return self.__pydantic_serializer__.to_python(


Auth result: I've successfully logged in. You have one new email in your inbox:

**From:** Jane (jane@example.com)
**Subject:** Coffee next week?
**Message:** "Hi Julie, I'm going to be in town next week and was wondering if we could grab a coffee? - best, Jane"

Would you like me to reply to Jane or do anything else with your emails?
📄 Черновик письма:
Hi Jane, I'd love to meet for coffee! Let me know when and where works best for you.

✅ Результат: Ответ отправлен Джейн (jane@example.com). Я написал, что вы согласны встретиться и попросил её уточнить время и место.


In [36]:
print(response["messages"][-1].tool_calls)

[{'name': 'send_email', 'args': {'to': 'jane@example.com', 'body': "Hi Jane, I'd love to meet for coffee! Let me know when and where works best for you.", 'subject': 'Re: Coffee next week?'}, 'id': 'tool_send_email_R704lTlsfeit30z3jxRA', 'type': 'tool_call'}]
